In [1]:
import datetime

import gym

from time import sleep
from tqdm import tqdm
import torch
from torch import nn
import copy
import random
from torch.utils.tensorboard import SummaryWriter

class Config():
    def __init__(self, lr = 0.01,
                gamma = 0.6,
                min_epsilon = 0.1,
                renew_target = 100,
                store_and_replay = True,
                double_dqn = True):
        self.lr = lr
        self.gamma = gamma
        self.min_epsilon = min_epsilon
        self.store_and_replay = store_and_replay
        self.double_dqn = double_dqn
        self.max_time_steps = 1000
        self.renew_target = renew_target
        
        self.replay_memory_fail = []
        self.replay_memory_fail_size = 1000
        self.replay_memory_success = []
        
        self.prev_success = set()
        
        dt = datetime.datetime.now()
        dt_str = dt.strftime("%Y-%m-%d %H-%M-%S")
        self.opt_str = f"SR_{store_and_replay}_DD_{double_dqn}"
        self.writer = SummaryWriter(f"runs/{self.opt_str}_{dt_str}")
        
        self.num_episodes = 5000
        
    def get_epsilon(self, episode = -1):
        if episode < 0:
            return self.min_epsilon
        return max(self.min_epsilon, 0.99**episode)
    
    def _insert_record(self, memory, rec, memory_size = -1):
        if rec in memory:
            return
        if len(memory) < memory_size or memory_size == - 1:
            memory.append(rec)
        else:
            memory[random.randint(0, memory_size - 1)] = rec
    
    def insert_record(self, rec):
        done = rec[-1]
        if done:
            self._insert_record(self.replay_memory_success, rec)
            self.prev_success.add(rec[0])
        else:
            self._insert_record(self.replay_memory_fail, rec, self.replay_memory_fail_size)
        
    def get_replay_memory(self):
        return self.replay_memory_success + self.replay_memory_fail
    
    def get_replay_record(self):
        replay_memory = self.get_replay_memory()
        mid_replay = random.randint(0, len(replay_memory) - 1)
        return mid_replay, replay_memory[mid_replay]
    

class QNet(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim = 16):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Embedding(num_states, 2*hidden_dim),
            nn.Linear(2 * hidden_dim, 2 * hidden_dim),
            nn.PReLU(),
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.PReLU(),
            nn.Linear(hidden_dim, num_actions)
        )

    def forward(self, x):
        x = self.layers(x)
        return x
        
def train(env, config, qnet):
    optimizer = torch.optim.SGD(qnet.parameters(), lr = 0.001)
    if config.double_dqn:
        qnet2 = copy.deepcopy(qnet)
    else:
        qnet2 = qnet
    
    criteria = nn.MSELoss()
    num_episodes = config.num_episodes
    
    for i in tqdm(range(1, num_episodes + 1)):
        train_episode(env, config, qnet, qnet2, optimizer, criteria, episode_count = i)
        if i % config.renew_target == 0:
            if config.double_dqn:
                qnet2 = copy.deepcopy(qnet)
                
        if i == 1 or i % 50 == 0:
            test(env, config, qnet, i)
            
    print("Training finished.\n")
    
def train_episode(env, config, qnet, qnet2, optimizer, criteria, episode_count = -1):
    done = False
    state = env.reset()
    n_steps = 0
    tot_reward = 0
    penalties = 0
    loss = 0
    
    while not done and n_steps < config.max_time_steps:
        state_t = torch.LongTensor([state])

        epsilon = config.get_epsilon(episode_count)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                q_hat = qnet(state_t)
                action = torch.argmax(q_hat[0]).item()

        next_state, reward, done, info = env.step(action)

        tot_reward += reward

        new_tuple = (state, action, next_state, reward, done)

        state = next_state
        if reward == -10:
            penalties += 1

        if config.store_and_replay:
            config.insert_record(new_tuple)
            loss_i, step_i = replay(qnet, qnet2, config, optimizer, criteria)
        else:
            loss_i, step_i = replay(qnet, qnet2, config, optimizer, criteria, rec = new_tuple)
        loss += loss_i
        n_steps += 1
        
def replay(qnet, qnet2, config, optimizer, criteria, num_instances = 5, rec = None):
    loss_i = 0
    
    for _ in range(num_instances):
        optimizer.zero_grad()
        if config.store_and_replay:
            mid_replay, rec = config.get_replay_record()
        
        state, action, next_state, reward, done = rec
        
        if done:
            y_t = torch.Tensor([reward])
        else:
            next_state_r_t = torch.LongTensor([next_state])
            with torch.no_grad():
                q_next = qnet2(next_state_r_t)
                
                y_t = reward + config.gamma * q_next.max(dim = -1)[0]
                
        state_r_t = torch.LongTensor([state])
        q_hat = qnet(state_r_t)
        q_hat = q_hat[:, action]
        
        loss = criteria(q_hat, y_t)
        loss.backward()
        optimizer.step()
        loss_i += loss.item()
        
    return loss_i, num_instances
        
    
def test(env, config, qnet, global_step = -1):
    qnet.eval()
    total_epochs, total_penalties = 0, 0
    episodes = 100
    
    total_reward = 0
    writer = config.writer
    
    for _ in tqdm(range(episodes)):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0
        
        done = False
        
        while not done and epochs < config.max_time_steps:
            with torch.no_grad():
                state_t = torch.LongTensor([state])
                q_hat = qnet(state_t)
                action = torch.argmax(q_hat[0]).item()
                
            state, reward, done, info = env.step(action)
            total_reward += reward
            if reward == -10:
                penalties += 1
            
            epochs += 1
            
        total_penalties += penalties
        total_epochs += epochs
    
    avg_steps = total_epochs / episodes
    avg_penalty = total_penalties / episodes
    avg_reward = total_reward / episodes
    
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {avg_steps}")
    print(f"Average penalty per episode: {avg_penalty}")
    print(f"Average reward per episode: {avg_reward}")
    
    if global_step > 0:
        writer.add_scalar("Steps", avg_steps, global_step)
        writer.add_scalar("Penalty", avg_penalty, global_step)
        writer.add_scalar("Reward", avg_reward, global_step)
    
if __name__ == "__main__":
    env = gym.make("Taxi-v3").env
    
    print("Action Space {}".format(env.action_space))
    print("State Space {}".format(env.observation_space))
    store_and_replay = True
    double_dqn = True
    
    print(f"Store & replay: {store_and_replay}")
    print(f"Double DQN: {double_dqn}")
    
    qnet = QNet(env.observation_space.n, env.action_space.n, hidden_dim = 32)
    config = Config(store_and_replay = store_and_replay, double_dqn = double_dqn, lr = 0.01, gamma = 0.75, renew_target = 20)
    
    train(env, config, qnet)
    test(env, config, qnet, config.num_episodes)

Action Space Discrete(6)
State Space Discrete(500)
Store & replay: False
Double DQN: False


  0%|          | 1/5000 [00:06<9:21:13,  6.74s/it]

Results after 100 episodes:
Average timesteps per episode: 1000.0
Average penalty per episode: 1000.0
Average reward per episode: -10000.0


  1%|          | 50/5000 [01:24<3:50:35,  2.79s/it]

Results after 100 episodes:
Average timesteps per episode: 1000.0
Average penalty per episode: 0.0
Average reward per episode: -1000.0


...

Results after 100 episodes:
Average timesteps per episode: 101.64
Average penalty per episode: 0.0
Average reward per episode: -82.53


100%|██████████| 5000/5000 [20:45<00:00,  4.01it/s]


Results after 100 episodes:
Average timesteps per episode: 180.67
Average penalty per episode: 0.0
Average reward per episode: -163.24
Training finished.



100%|██████████| 100/100 [00:00<00:00, 114.28it/s]

Results after 100 episodes:
Average timesteps per episode: 170.75
Average penalty per episode: 0.0
Average reward per episode: -153.11
